In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
import datetime as dt
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName("appName").setMaster("local")
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [29]:
x=sc.textFile("hdfs://172.16.1.114:8020/user/hive/warehouse/workingDirectory/500000_Records.csv",minPartitions = 4)
#x=(sc.textFile("hdfs://172.16.1.114:8020/user/hive/warehouse/workingDirectory/500000_Records.csv",minPartitions = 4).map(lambda x: x.split(",")))

In [30]:
x.take(2) # top N records 
x.getNumPartitions()  ## Number of Partitions
#x.count() # counting the number of records or line in the file 

1

In [5]:
def Func(lines):
    lines = lines.lower()
    lines = lines.split(',')
    return lines

In [6]:
flatmapped = x.flatMap(lambda x: x.split(','))
mapped =  flatmapped.map(lambda x:(x,1))
grouped = mapped.groupByKey()
group_feq = grouped.mapValues(sum).map(lambda x: (x[1],x[0])).sortByKey(False)
#group_feq.take(100)
distinct1 = mapped.distinct()
distinct1.count()

4130827

In [5]:
flatmapped = x.flatMap(lambda x: x.split(','))
sample1 = flatmapped.sample(False,2000)
sample1.count()

18500037

In [8]:
### Joins 
a = sc.parallelize([('a',2),('b',3)])
b = sc.parallelize([('a',4),('b',5),('c',6)])
c = b.join(a)
c.take(2)

[('b', (5, 3)), ('a', (4, 2))]

In [9]:
num_rdd = sc.parallelize(range(1,50000))
num_rdd.reduce(lambda x,y:x+y)

1249975000

In [15]:
test= [('a',1),('b',2),('a',3),('d',4),('e',5)]  

sc.parallelize(test).sortByKey(True,1).collect()  ## arranges in asc or desc order the input shoule be keyvalue order by key

[('a', 1), ('a', 3), ('b', 2), ('d', 4), ('e', 5)]

In [28]:
rdd1 = sc.parallelize([1,2,3,4])
rdd2 = sc.parallelize([4,5,6,7])

rdd1.union(rdd2).collect()  # union 
rdd1.intersection(rdd2).collect() # intersection
rdd1.subtract(rdd2).collect() # substract
rdd1.cartesian(rdd2).collect()  # cartesian


[(1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (3, 4),
 (3, 5),
 (3, 6),
 (3, 7),
 (4, 4),
 (4, 5),
 (4, 6),
 (4, 7)]